In [5]:
import json
import os
import csv
import pandas as pd
from nltk.corpus import stopwords


In [9]:

# Fungsi untuk membaca kamus slang
def load_slang_dict(path):
    """Memuat kamus kata slang dari file JSON"""
    try:
        with open(path, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"Kamus slang tidak ditemukan di {path}")
        return {}

# Fungsi untuk memuat stopwords kustom
def load_custom_stopwords(path):
    """Memuat stopwords kustom dari file"""
    try:
        with open(path, 'r') as f:
            return f.read().splitlines()
    except FileNotFoundError:
        print(f"File stopwords tidak ditemukan di {path}")
        return []

# Fungsi untuk mengganti kata slang di teks berdasarkan kamus slang
def replace_slang_in_text(text, slang_dict):
    """Mengganti kata slang dalam teks berdasarkan kamus slang"""
    return [" ".join(slang_dict.get(word, word) for word in line.split()) for line in text]

# Fungsi untuk menghapus stopwords dari teks
def remove_stopwords_from_text(text, stopwords_list):
    """Menghapus stopwords dari teks"""
    return [" ".join(word for word in line.split() if word not in stopwords_list) for line in text]

# Fungsi untuk menyimpan hasil ke file CSV
def save_to_csv(df, output_file):
    """Menyimpan DataFrame ke file CSV"""
    df.to_csv(output_file, index=False)
    print(f"Hasil disimpan di {output_file}")

# Fungsi utama untuk menjalankan semua langkah dalam pipeline
def process_text_pipeline(input_csv, slang_file, stopwords_file):
    """Pipeline utama untuk memproses teks, mengganti slang, dan menghapus stopwords"""
    
    # Langkah 1: Memuat kamus slang
    slang_dict = load_slang_dict(slang_file)

    # Langkah 2: Memuat data dari CSV
    df = pd.read_csv(input_csv)
    
    # Gunakan kolom 'text_clean' sebagai input teks
    text = df['text_stopword'].fillna('')  # Pastikan tidak ada nilai NaN

    # Langkah 3: Mengganti kata slang di teks
    text_slang_replaced = replace_slang_in_text(text, slang_dict)
    df['text_slang'] = text_slang_replaced  # Simpan hasil penggantian slang ke kolom baru

    # Langkah 4: Memuat stopwords kustom dan NLTK
    custom_stopwords = load_custom_stopwords(stopwords_file)
    stopwords_list = stopwords.words('indonesian')
    stopwords_list.extend(custom_stopwords)

    # Langkah 5: Menghapus stopwords dari teks
    text_no_stopwords = remove_stopwords_from_text(text_slang_replaced, stopwords_list)
    df['text_no_stopwords'] = text_no_stopwords  # Simpan hasil penghapusan stopwords ke kolom baru

    # Simpan hasil ke CSV
    output_file = os.path.splitext(input_csv)[0] + '_processed.csv'
    save_to_csv(df, output_file)



In [10]:
# Path file input dan kamus
input_csv = '03_hasil-stopword-filter.csv'  # Input file yang diunggah
slang_file = "kamus/nasalsabila_kamus-alay/_json_colloquial-indonesian-lexicon.txt"
stopwords_file = 'kamus/masdevid_id-stopwords/id.stopwords.02.01.2016.txt'

# Menjalankan pipeline
process_text_pipeline(input_csv, slang_file, stopwords_file)


Hasil disimpan di 03_hasil-stopword-filter_processed.csv
